# Get Historical Market Cap Data

https://site.financialmodelingprep.com/developer/docs#historical-market-cap-company-information

format: https://financialmodelingprep.com/api/v3/historical-market-capitalization/AAPL?limit=100&from=2023-10-10&to=2023-12-10

each query is 5 years only

In [3]:
import pandas as pd
import requests

# Load the CSV file into a pandas DataFrame
df = pd.read_csv("~/Box/STAT 222 Capstone/Intermediate Data/Credit_Rating/combined_credit_rating_data.csv")

# Extract all distinct "Symbol" values
distinct_symbols = df["Symbol"].unique()

# Convert the array of distinct symbols to a pandas list
distinct_symbols_list = distinct_symbols.tolist()

print(len(distinct_symbols_list))

886


In [2]:
# API key for accessing the financial data
api_key = "GzzXB8zIm8u1NJzcFKZNOwcOyxarZEmi"

# # Function to fetch financial statements data
# def fetch_financial_statements(company):
#     # Number of years for which data will be retrieved
#     quarters = 67
    
#     # Fetching balance sheet data
#     balance_sheet = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?period=quarter&limit={quarters}&apikey={api_key}')
#     balance_sheet = balance_sheet.json()

#     # Fetching income statement data
#     income_statement = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?period=quarter&limit={quarters}&apikey={api_key}')
#     income_statement = income_statement.json()
    
#     # Fetching cash flow statement data
#     cash_flow_statement = requests.get(f'https://financialmodelingprep.com/api/v3/cash-flow-statement/{company}?period=quarter&limit={quarters}&apikey={api_key}')
#     cash_flow_statement = cash_flow_statement.json()

#     return balance_sheet, income_statement, cash_flow_statement